In [4]:
import pandas as pd
import glob
import os

In [17]:
def combine_csv_values(input_directory, output_file):
    """
    Combines the 'Value' columns from multiple CSV files into a single CSV file.

    Parameters:
    - input_directory: str, path to the directory containing input CSV files.
    - output_file: str, path for the output combined CSV file.
    """

    # Initialize an empty DataFrame
    combined_df = pd.DataFrame()

    # Get list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(input_directory, '*.csv'))

    if not csv_files:
        print(f"No CSV files found in the directory: {input_directory}")
        return

    for file in csv_files:
        try:
            # Read the CSV file
            df = pd.read_csv(file)

            # Ensure the necessary columns exist
            required_columns = {'ISO3', 'Value', 'Date'}
            if not required_columns.issubset(df.columns):
                print(f"Skipping file '{file}' due to missing columns.")
                continue

            # Get the ISO3 code (assuming it's the same for all rows)
            iso3 = df['ISO3'].iloc[0]

            # Select Date and Value columns
            df_subset = df[['Date', 'Value']].copy()

            # Convert Date to datetime if not already
            df_subset['Date'] = pd.to_datetime(df_subset['Date'])

            # Convert Value to float, coercing errors to NaN
            df_subset['Value'] = pd.to_numeric(df_subset['Value'], errors='coerce')

            # Set Date as index
            df_subset.set_index('Date', inplace=True)

            # Rename the Value column to ISO3 code
            df_subset.rename(columns={'Value': iso3}, inplace=True)

            # Merge with the combined DataFrame
            if combined_df.empty:
                combined_df = df_subset
            else:
                combined_df = combined_df.join(df_subset, how='outer')

        except Exception as e:
            print(f"Error processing file '{file}': {e}")
            continue

    if combined_df.empty:
        print("No data to combine.")
        return

    # Sort by Date
    combined_df.sort_index(inplace=True)

    # Reset index to have Date as a column
    combined_df.reset_index(inplace=True)

    # Optional: Rename 'index' to 'Date' if necessary
    combined_df.rename(columns={'index': 'Date'}, inplace=True)

    # Save to CSV
    try:
        # Define float format
        float_format = '%.6f'

        combined_df.to_csv(output_file, index=False, float_format=float_format)
        print(f"Combined CSV saved to '{output_file}'")
    except Exception as e:
        print(f"Failed to save the combined CSV: {e}")

In [23]:
# Convert all CSV files in main directory
if __name__ == "__main__":
    # Specify the directory containing CSV files
    input_dir = ''

    # Specify the output file path
    output_csv = 'combined_values.csv'

    combine_csv_values(input_dir, output_csv)

Skipping file 'combined_values.csv' due to missing columns.
Combined CSV saved to 'combined_values.csv'
